<a href="https://colab.research.google.com/github/samuramirez/cellmigration/blob/master/VAMPIRE_Google_Colab/VAMPIRE_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import auth
#This allows SDK to see and edit Google Drive files
#SDK is required to interact with GCP
auth.authenticate_user()

In [ ]:
#Get from Drive the folder libraries
!cp -r /content/drive/MyDrive/SourceCode /content

from SourceCode.mainbody import mainbody
from SourceCode.getboundary import getboundary

In [ ]:
import csv
import ntpath

# Copy segmented, labeled images from GCP to local directory

In [ ]:
maskspath1 = 'cellmbucket/analysis_eliz_05262021/trial1/052622021_Fascin_KO_sample4_trackedmasks.zip' #@param {type:"string"}
maskspath2 = 'cellmbucket/analysis_eliz_05262021/trial1/052622021_Fascin_KO_sample3_trackedmasks.zip' #@param {type:"string"}

In [ ]:
!gsutil cp gs://{maskspath1} /content
!gsutil cp gs://{maskspath2} /content

Copying gs://cellmbucket/analysis_eliz_05262021/trial1/052622021_Fascin_KO_sample4_trackedmasks.zip...
/ [1 files][  2.4 MiB/  2.4 MiB]                                                
Operation completed over 1 objects/2.4 MiB.                                      
Copying gs://cellmbucket/analysis_eliz_05262021/trial1/052622021_Fascin_KO_sample3_trackedmasks.zip...
/ [1 files][  2.4 MiB/  2.4 MiB]                                                
Operation completed over 1 objects/2.4 MiB.                                      


In [ ]:
!unzip {ntpath.basename(maskspath1)}
!unzip {ntpath.basename(maskspath2)}

Archive:  052622021_Fascin_KO_sample4_trackedmasks.zip
   creating: 052622021_Fascin_KO_sample4_trackedmasks/
  inflating: 052622021_Fascin_KO_sample4_trackedmasks/052622021_Sample4_w1DIC_s21_t2.TIF  
  inflating: 052622021_Fascin_KO_sample4_trackedmasks/052622021_Sample4_w1DIC_s18_t38.TIF  
  inflating: 052622021_Fascin_KO_sample4_trackedmasks/052622021_Sample4_w1DIC_s10_t116.TIF  
  inflating: 052622021_Fascin_KO_sample4_trackedmasks/052622021_Sample4_w1DIC_s24_t77.TIF  
  inflating: 052622021_Fascin_KO_sample4_trackedmasks/052622021_Sample4_w1DIC_s21_t120.TIF  
  inflating: 052622021_Fascin_KO_sample4_trackedmasks/052622021_Sample4_w1DIC_s3_t134.TIF  
  inflating: 052622021_Fascin_KO_sample4_trackedmasks/052622021_Sample4_w1DIC_s8_t73.TIF  
  inflating: 052622021_Fascin_KO_sample4_trackedmasks/052622021_Sample4_w1DIC_s32_t123.TIF  
  inflating: 052622021_Fascin_KO_sample4_trackedmasks/052622021_Sample4_w1DIC_s4_t157.TIF  
  inflating: 052622021_Fascin_KO_sample4_trackedmasks/0526220

# Make csv file that contains paths to segmented, labeled images in local directory with descriptions for each set

In [ ]:
csv_fields=['set ID', 'condition', 'set location', 'tag', 'note']

In [ ]:
#@markdown Parameters for the csv that indicate where the masks are located

#@markdown Row index number
setID1 = '1' #@param {type:"string"}
setID2 = '2' #@param {type:"string"}
#@markdown Description of an image set
condition1 = 'FascinKO' #@param {type:"string"}
condition2 = 'control' #@param {type:"string"}
#@markdown The path of the folder containing the segmented images
set_location1 = '/content/052622021_Fascin_KO_sample4_trackedmasks' #@param {type:"string"}
set_location2 = '/content/052622021_Fascin_KO_sample3_trackedmasks' #@param {type:"string"}
#@markdown String of text where only segmented images in the set location with filenames containing the tag will be analyzed
tag1 = 'DIC' #@param {type:"string"}
tag2 = 'DIC' #@param {type:"string"}
#@markdown Any information about the image sets used only for the user's record. This is not used in the analysis
note1 = 'trial1sample4' #@param {type:"string"}
note2 = 'trial1sample3' #@param {type:"string"}
#@markdown Name of the csv file that is created
csv_name = 'segmented_image_sets_to_build_model_colab' #@param {type:"string"}

In [ ]:
rows=[[setID1,
  condition1,
  set_location1,
  tag1,
  note1],
 [setID2,
  condition2,
  set_location2,
  tag2,
  note2]]

In [ ]:
csv_filename = '/content/' + csv_name + '.csv'

In [ ]:
with open(csv_filename, 'w') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)
      
    # writing the fields
    csvwriter.writerow(csv_fields)
      
    # writing the data rows
    csvwriter.writerows(rows)

# Specify VAMPIRE model parameters

In [ ]:
entries={}

In [ ]:
fields = (
        'Build Model', 'Image sets to build', 'Number of coordinates', 'Number of shape modes', 'Model output folder',
        'Model name', 'Image sets to apply', 'Model to apply', 'Result output folder')

In [ ]:
#@markdown Parameters for VAMPIRE model

#@markdown Build model (True or False)
Build_Model = False #@param {type:"boolean"}
#@markdown Path to csv file to load that contains path to masks
Image_sets_to_build = '/content/segmented_image_sets_to_build_model_colab.csv' #@param {type:"string"}
#@markdown Number of coordinates used to describe cell shape
Number_of_coordinates = '50' #@param {type:"string"}
#@markdown Number of clusters to form to describe cell shapes
Number_of_shape_modes = '5' #@param {type:"string"}
#@markdown Path to location where output data is saved
Model_output_folder = '/content' #@param {type:"string"}
#@markdown Name of folder where output will be saved (and name of the model pickle file)
Model_name = '50Coords5ShapeModes' #@param {type:"string"}
#@markdown Apply model

#@markdown Only complete the below entries if you are applying the model

#@markdown Path to csv file to load that contains path to masks
Image_sets_to_apply = '/content/segmented_image_sets_to_build_model_colab.csv' #@param {type:"string"}
#@markdown Path to pickle model file 
Model_to_apply = '/content/50Coords5ShapeModes/50Coords5ShapeModes.pickle' #@param {type:"string"}
#@markdown Path to location where outpur data is saved (usually the folder that was created with Build Model)
Result_output_folder = '/content/50Coords5ShapeModes' #@param {type:"string"}


In [ ]:
ents = [Build_Model, Image_sets_to_build, Number_of_coordinates, Number_of_shape_modes, Model_output_folder, Model_name, Image_sets_to_apply, Model_to_apply, Result_output_folder]

for count,field in enumerate(fields):
  entries[field] = ents[count]

In [ ]:
# Make folders in local directory where figures and model pickle file will be saved 

if Build_Model:
  foldername = Model_output_folder + '/' + Model_name
  !mkdir {foldername}
  newfolder = foldername + '/' + 'Example model figures'
  !mkdir '{newfolder}'
else:
  foldername = Result_output_folder + '/' + 'Result based on ' + ntpath.basename(Result_output_folder)
  !mkdir '{foldername}'

mkdir: cannot create directory ‘/content/50Coords5ShapeModes/Result based on 50Coords5ShapeModes’: File exists


In [ ]:
entries

{'Build Model': False,
 'Image sets to apply': '/content/segmented_image_sets_to_build_model_colab.csv',
 'Image sets to build': '/content/segmented_image_sets_to_build_model_colab.csv',
 'Model name': '50Coords5ShapeModes',
 'Model output folder': '/content',
 'Model to apply': '/content/50Coords5ShapeModes/50Coords5ShapeModes.pickle',
 'Number of coordinates': '50',
 'Number of shape modes': '5',
 'Result output folder': '/content/50Coords5ShapeModes'}

In [ ]:
def Model(entries, buildModel):
    if buildModel:
        csv = entries['Image sets to build']
        clnum = entries['Number of shape modes']
        outpth = entries['Model output folder']  # name
        getboundary(csv, outpth, entries, buildModel)  # create registry csv and boundary stack
        mainbody(buildModel, csv, entries, outpth, clnum)
    else:
        csv = entries['Image sets to apply']
        outpth = entries['Result output folder']
        clnum = None
        getboundary(csv, outpth, entries, buildModel)  # create registry csv and boundary stack
        mainbody(buildModel, csv, entries, outpth, clnum)


def vampire():
    Model(entries, Build_Model)

# Run the VAMPIRE analysis

In [ ]:
vampire()

## getboundary.py
registry or boundary already exist
registry or boundary already exist
## main.py
## bdreg.py
applying model
available cpu cores :  2
For parallel of bdreg, elapsed time is 19.189849853515625seconds...
## pca_bdreg.py
For PCA bdreg, elapsed time is 0.010967254638671875seconds...
# clusterSM
For cluster, elapsed time is 0.5561351776123047seconds...
## update_csv.py
## bdreg.py
applying model
available cpu cores :  2
For parallel of bdreg, elapsed time is 19.4113347530365seconds...
## pca_bdreg.py
For PCA bdreg, elapsed time is 0.0048677921295166016seconds...
# clusterSM
For cluster, elapsed time is 0.554415225982666seconds...
## update_csv.py


# Copy files generated from VAMPIRE analysis (csv files of shape features, model pickle file, pickle files of boundary coordinates, and figures) to GCP bucket

In [ ]:
#@markdown Where to save VAMPIRE files in GCP bucket
GCP_path = 'cellmbucket/analysis_eliz_05262021/trial1' #@param {type:"string"}
#@markdown Name of folder that will contain all generated files from the VAMPIRE analysis in GCP bucket
output_folder = '052622021_Fascin_KO_VAMPIRE_analysis' #@param {type:"string"}

In [ ]:
output_data = Model_output_folder + '/' + Model_name

output_path = GCP_path + '/' + output_folder + '/'

!gsutil cp -r '{output_data}'  gs://{output_path}

Copying file:///content/50Coords5ShapeModes/VAMPIRE datasheet control.csv [Content-Type=text/csv]...
Copying file:///content/50Coords5ShapeModes/FascinKO_boundary_coordinate_stack.pickle [Content-Type=application/octet-stream]...
Copying file:///content/50Coords5ShapeModes/VAMPIRE datasheet FascinKO.csv [Content-Type=text/csv]...
Copying file:///content/50Coords5ShapeModes/control_boundary_coordinate_stack.pickle [Content-Type=application/octet-stream]...
-
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file:///content/50Coords5ShapeModes/50Coords5ShapeModes.pickle [Content-Type=application/octet-stream]...
Copying file:///content/50Coords5ShapeModes/Result based on 50Coords5ShapeModes/Shape mode distribution_1_FascinKO.png [Content-Type=image/png]...
Copying file:///content/5